# **Extract GridMET Data**

By Bridget Bittmann

Date created: March 28, 2022

Date editted: 

Purpose: This script extracts GridMET data from Google Earth Engine using the Earth Engine API in Python.

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=26Ko4Y7LomSorTrhPWwGAx4LN1OU-wYtYNhgjqTM6yE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiRiBQc8iA2nwdp-Tn1op3ff28jRyOS1KOFjrZU_KUVeEywXcCHrm8

Successfully saved authorization token.


In [2]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile

     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 16.7 MB 327 kB/s 


In [3]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Go to correct folder in Google Drive
%cd drive/MyDrive/spatial_colab/datasets/
%ls

/content/drive/MyDrive/spatial_colab/datasets
2021_clip/      CDL_subsets/           irrig_lbrb/  lcmap_files/
Boise_CDL/      diversion_timeseries/  IrrMapper/   masked/
CDL_reproject/  irrigation_companies/  LBRB_shp/    subset_test_shp/


In [ ]:

# Algorithm:
# 1. Import shapefile of chosing
# 3. Create a new feature with selected watershed
# 5. Union the watersheds (if necessary), export the watershed(s)
# 6. Get the CDL data
# 7. Clip to the area of the watershed
# 8. Export 

#--------------------------------#
#   Name           |    huc8     #
#--------------------------------#
#   Lower Boise    |  17050114   #
#--------------------------------#




In [23]:
## ----------------------------------- ## 
## 1. Import shapefile to clip dataset ##
## ----------------------------------- ## 

shp_file = 'subset_test_shp/subset.shp'
subset = emap.shp_to_ee(shp_file)

map=emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(subset, 0, 2), {}, 'POU')
map.addLayerControl()
map

In [24]:
## -------------------------- ## 
## 2. Import the GridMET data ##
## -------------------------- ## 

dataset = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('1987-01-01', '2020-12-31')
dataset.first()
max_temp = dataset.select('tmmx') \
            .map(lambda image:image.clip(subset))

max_tempVis = {
    'min': 290.0,
    'max': 314.0,
    'palette': ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c']

}

map = emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(max_temp , max_tempVis,"Maximum Temperature")

map.addLayerControl()
map


In [36]:
## ---------------------- ##
## IMPORT THE DAYMET DATA ##
## ---------------------- ##

daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate('1987-01-01', '1987-12-31')

mxtmp = daymet.select('tmax').map(lambda image: image.clip(subset))

maximumTemperatureVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(mxtmp, maximumTemperatureVis, 'tmax')
Map

In [38]:
## ------------------------------ ## 
## CALCULATE STATS ON DAYMET CODE ##
## ------------------------------ ## 

mean_max = mxtmp.mean()

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(mean_max, maximumTemperatureVis, 'tmax')
Map

In [51]:
## --------------------- ##
## CALCULATE ZONAL STATS ##
## --------------------- ##

import os

out_stats = os.path.join('climate_stats/stats.csv')

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
# emap.zonal_statistics(dem, states, out_dem_stats, statistics_type='MEAN', scale=1000)

emap.zonal_statistics(mean_max, subset, out_stats, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/stats.csv
